In [1]:
# Bibliotecas essenciais para processamento distribuído com PySpark
# Funções específicas para transformação de dados e criação de IDs
# Definição de tipos de dados para garantir integridade do schema

from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast, monotonically_increasing_id, year, month, dayofmonth, date_format, col, count, when, isnan, expr, lower
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType, DoubleType
import os

In [2]:
# Inicialização do Spark com configuração local para processamento dos dados de vendas

spark = SparkSession.builder.appName("Modelagem Dimensional - Vendas").master("local[*]").getOrCreate()

In [3]:
BASE_PATH = "dados"
INPUT_PATH = os.path.join(BASE_PATH, "entrada")
OUTPUT_PATH = os.path.join(BASE_PATH, "saida")

### Definição do schema e carregamento dos dados brutos

In [4]:
# Schema explícito para evitar inferências incorretas de tipos
# Campos definidos como não-nullable (False) para garantir integridade

schema = StructType([
    StructField("nome_cliente", StringType(), False),
    StructField("cidade", StringType(), False),
    StructField("estado", StringType(), False), 
    StructField("nome_produto", StringType(), False),
    StructField("categoria", StringType(), False),
    StructField("fabricante", StringType(), False),
    StructField("data", DateType(), False),
    StructField("qtd_vendida", IntegerType(), False),
    StructField("valor_total", DoubleType(), False)
])

In [5]:
# Carregamento dos dados brutos com schema predefinido

df_bruto = spark.read.csv(os.path.join(INPUT_PATH,"dados_brutos.csv"), header=True, schema=schema)

In [6]:
df_bruto.show(n=5)
df_bruto.printSchema()

+-------------+--------------+------+------------+---------+----------+----------+-----------+-----------+
| nome_cliente|        cidade|estado|nome_produto|categoria|fabricante|      data|qtd_vendida|valor_total|
+-------------+--------------+------+------------+---------+----------+----------+-----------+-----------+
|Lucas Pereira|  Porto Alegre|    RS|  Detergente|  Limpeza|       Ypê|2024-01-26|          6|       90.0|
|Lucas Pereira|  Porto Alegre|    RS|      Feijão| Alimento|   Kicaldo|2024-01-14|         10|      240.0|
| Ana Oliveira|Rio de Janeiro|    RJ|Refrigerante|   Bebida| Coca-Cola|2024-01-15|          3|      150.0|
| Pedro Santos|      Curitiba|    PR|      Feijão| Alimento|   Kicaldo|2024-01-28|          4|      152.0|
| Pedro Santos|      Curitiba|    PR|       Arroz| Alimento|     Camil|2024-01-24|          3|       87.0|
+-------------+--------------+------+------------+---------+----------+----------+-----------+-----------+
only showing top 5 rows

root
 |-- no

In [7]:
# Renomeação de colunas para seguir padrão dimensional com prefixo 'nome_'
# Importante para clareza e consistência no modelo dimensional

colunas_renomeacao = {
    "cidade": "nome_cidade",
    "estado": "nome_estado",
    "categoria": "nome_categoria",
    "fabricante": "nome_fabricante"
}

for nome_antigo, nome_novo in colunas_renomeacao.items():
    df_bruto = df_bruto.withColumnRenamed(nome_antigo, nome_novo)

In [8]:
# Separação de colunas por tipo para facilitar análises específicas

colunas_numericas = [nome for nome, tipo in df_bruto.dtypes if tipo in ('int', 'double', 'float', 'long', 'short', 'decimal')]
colunas_categoricas = [nome for nome, tipo in df_bruto.dtypes if tipo in ('string', 'boolean')]

print("Colunas numéricas:", colunas_numericas)
print("Colunas categóricas:", colunas_categoricas)

Colunas numéricas: ['qtd_vendida', 'valor_total']
Colunas categóricas: ['nome_cliente', 'nome_cidade', 'nome_estado', 'nome_produto', 'nome_categoria', 'nome_fabricante']


### Verificação das estatísticas descritivas, inconsistências e valores ausentes

In [9]:
# Estatísticas descritivas dos dados numéricos para validação inicial

df_bruto[colunas_numericas].describe().show()

+-------+------------------+------------------+
|summary|       qtd_vendida|       valor_total|
+-------+------------------+------------------+
|  count|               100|               100|
|   mean|              5.58|            148.44|
| stddev|2.8645517215886613|108.93277697272707|
|    min|                 1|               9.0|
|    max|                10|             470.0|
+-------+------------------+------------------+



In [10]:
# Função robusta para detectar valores nulos conforme o tipo de dado
# Cálculo de percentual de nulos para avaliação de completude
# Retorno booleano para facilitar integração com pipelines de validação

def validar_qualidade_dados(df, nome_coluna):
    """Método para validar a qualidade dos dados em uma coluna (baseado apenas na quantidade de valores nulos)"""
    coluna_dtype = dict(df.dtypes)[nome_coluna]
    
    if coluna_dtype in ['float', 'double', 'decimal', 'int', 'bigint', 'smallint', 'tinyint']:
        condicao = col(nome_coluna).isNull() | isnan(col(nome_coluna)) | (col(nome_coluna) == "")
    else:
        condicao = col(nome_coluna).isNull() | (col(nome_coluna) == "")
    
    contagem_nulos = df.filter(condicao).count()
    contagem_total = df.count()
    percentual_nulos = (contagem_nulos / contagem_total) * 100 if contagem_total > 0 else 0
    
    print(f"Coluna {nome_coluna}: {contagem_nulos} valores nulos ({percentual_nulos:.2f}%)")
    return contagem_nulos == 0

In [11]:
# Verificação de valores nulos em todas as colunas do dataset

for coluna in df_bruto.columns:
    validar_qualidade_dados(df_bruto, coluna)

Coluna nome_cliente: 0 valores nulos (0.00%)
Coluna nome_cidade: 0 valores nulos (0.00%)
Coluna nome_estado: 0 valores nulos (0.00%)
Coluna nome_produto: 0 valores nulos (0.00%)
Coluna nome_categoria: 0 valores nulos (0.00%)
Coluna nome_fabricante: 0 valores nulos (0.00%)
Coluna data: 0 valores nulos (0.00%)
Coluna qtd_vendida: 0 valores nulos (0.00%)
Coluna valor_total: 0 valores nulos (0.00%)


In [12]:
# Normalização de casing para garantir consistência nas dimensões
# Estratégia fundamental para evitar duplicidades nas dimensões

df_bruto = df_bruto.withColumn("nome_cliente", expr("initcap(nome_cliente)"))
df_bruto = df_bruto.withColumn("nome_cidade", expr("initcap(nome_cidade)"))
df_bruto = df_bruto.withColumn("nome_estado", expr("upper(nome_estado)"))
df_bruto = df_bruto.withColumn("nome_produto", expr("initcap(nome_produto)"))
df_bruto = df_bruto.withColumn("nome_categoria", expr("initcap(nome_categoria)"))
df_bruto = df_bruto.withColumn("nome_fabricante", expr("initcap(nome_fabricante)"))

In [13]:
df_bruto.show(10)

+-------------+--------------+-----------+------------+--------------+---------------+----------+-----------+-----------+
| nome_cliente|   nome_cidade|nome_estado|nome_produto|nome_categoria|nome_fabricante|      data|qtd_vendida|valor_total|
+-------------+--------------+-----------+------------+--------------+---------------+----------+-----------+-----------+
|Lucas Pereira|  Porto Alegre|         RS|  Detergente|       Limpeza|            Ypê|2024-01-26|          6|       90.0|
|Lucas Pereira|  Porto Alegre|         RS|      Feijão|      Alimento|        Kicaldo|2024-01-14|         10|      240.0|
| Ana Oliveira|Rio De Janeiro|         RJ|Refrigerante|        Bebida|      Coca-cola|2024-01-15|          3|      150.0|
| Pedro Santos|      Curitiba|         PR|      Feijão|      Alimento|        Kicaldo|2024-01-28|          4|      152.0|
| Pedro Santos|      Curitiba|         PR|       Arroz|      Alimento|          Camil|2024-01-24|          3|       87.0|
| Pedro Santos|      Cur

### Identificação das dimensões e fato

A identificação das dimensões (Cliente, Produto, Local e Data) e do fato (Vendas) na modelagem dimensional seguiu a metodologia de Kimball, onde foram avaliados os elementos de análise do negócio separando claramente os descritores contextuais (dimensões) das métricas mensuráveis (fatos). As dimensões representam os diferentes contextos de análise (quem comprou, o que foi comprado, onde e quando ocorreu a compra), permitindo filtros e agrupamentos, enquanto a tabela fato armazena apenas as métricas numéricas (quantidade vendida e valor total) e as chaves para as dimensões. Esta estrutura em estrela otimiza consultas analíticas ao reduzir junções complexas e facilitar a navegação multidimensional dos dados de vendas.

![Diagrama da Modelagem](diagrama.png)

### Criação das tabelas dimensões e fato (modelagem dimensional)

In [14]:
class ModelagemDimensional:
    def __init__(self, spark_session):
        self.spark = spark_session
        self.dimensoes = {}
        self.fatos = {}
        
    def extrair_dimensao(self, df_origem, nome_dimensao, colunas, chave_natural=None):
        """Método para extrair dimensões"""
        dim = df_origem.select(*colunas).distinct()
        chave_sk = f"id_{nome_dimensao}"
        dim = dim.withColumn(chave_sk, monotonically_increasing_id())
        self.dimensoes[nome_dimensao] = dim
        print(f"Dimensão {nome_dimensao} criada: {dim.count()} registros")
        return dim
    
    def construir_fato(self, df_origem, nome_fato, joins_dimensoes, metricas):
        """Método para construção de fatos"""
        df_atual = df_origem.withColumn(f"id_{nome_fato}", monotonically_increasing_id())
        
        for dim_nome, configuracao in joins_dimensoes.items():
            colunas_select = configuracao["colunas_join"] + [configuracao["chave_sk"]]
            
            df_atual = df_atual.join(
                self.dimensoes[dim_nome].select(*colunas_select),
                on=configuracao["colunas_join"],
                how="left"
            )
        
        colunas_finais = [f"id_{nome_fato}"] + [config["chave_sk"] for config in joins_dimensoes.values()] + metricas
        self.fatos[nome_fato] = df_atual.select(*colunas_finais)
        return self.fatos[nome_fato]

In [15]:
modelo = ModelagemDimensional(spark)

In [16]:
dim_cliente = modelo.extrair_dimensao(df_origem=df_bruto, nome_dimensao="cliente", colunas=["nome_cliente"])
dim_produto = modelo.extrair_dimensao(df_origem=df_bruto, nome_dimensao="produto", colunas=["nome_produto", "nome_categoria", "nome_fabricante"])
dim_local = modelo.extrair_dimensao(df_origem=df_bruto, nome_dimensao="local", colunas=["nome_cidade", "nome_estado"])
dim_data = modelo.extrair_dimensao(df_origem=df_bruto, nome_dimensao="data", colunas=["data"])

Dimensão cliente criada: 5 registros
Dimensão produto criada: 5 registros
Dimensão local criada: 5 registros
Dimensão data criada: 29 registros


In [17]:
# Enriquecimento da Dimensão Data

dim_data = dim_data.withColumn("ano", year("data")) \
    .withColumn("mes", month("data")) \
    .withColumn("dia", dayofmonth("data")) \
    .withColumn("dia_semana", date_format("data", "EEEE"))

modelo.dimensoes["data"] = dim_data

In [18]:
for nome, dim in modelo.dimensoes.items():
    print(f"\nDimesão {nome}:")
    dim.show(5)


Dimesão cliente:
+-------------+----------+
| nome_cliente|id_cliente|
+-------------+----------+
| Ana Oliveira|         0|
| Pedro Santos|         1|
|Lucas Pereira|         2|
|   João Silva|         3|
|  Maria Souza|         4|
+-------------+----------+


Dimesão produto:
+------------+--------------+---------------+----------+
|nome_produto|nome_categoria|nome_fabricante|id_produto|
+------------+--------------+---------------+----------+
| Sabão Em Pó|       Limpeza|            Omo|         0|
|  Detergente|       Limpeza|            Ypê|         1|
|       Arroz|      Alimento|          Camil|         2|
|Refrigerante|        Bebida|      Coca-cola|         3|
|      Feijão|      Alimento|        Kicaldo|         4|
+------------+--------------+---------------+----------+


Dimesão local:
+--------------+-----------+--------+
|   nome_cidade|nome_estado|id_local|
+--------------+-----------+--------+
|      Curitiba|         PR|       0|
|     São Paulo|         SP|       1|


#### Fato Vendas

In [19]:
joins_configuracao = {
    "cliente": {
        "colunas_join": ["nome_cliente"],
        "chave_sk": "id_cliente"
    },
    "produto": {
        "colunas_join": ["nome_produto", "nome_categoria", "nome_fabricante"],
        "chave_sk": "id_produto"
    },
    "local": {
        "colunas_join": ["nome_cidade", "nome_estado"],
        "chave_sk": "id_local"
    },
    "data": {
        "colunas_join": ["data"],
        "chave_sk": "id_data"
    }
}

metricas = ["qtd_vendida", "valor_total"]

fato_vendas = modelo.construir_fato(df_origem=df_bruto, nome_fato="venda", joins_dimensoes=joins_configuracao, metricas=metricas)

print(f"Tabela fato Vendas criada: {fato_vendas.count()} registros")
fato_vendas.orderBy("id_venda").show()

Tabela fato Vendas criada: 100 registros
+--------+----------+----------+--------+-------+-----------+-----------+
|id_venda|id_cliente|id_produto|id_local|id_data|qtd_vendida|valor_total|
+--------+----------+----------+--------+-------+-----------+-----------+
|       0|         2|         1|       2|     11|          6|       90.0|
|       1|         2|         4|       2|      3|         10|      240.0|
|       2|         0|         3|       4|     15|          3|      150.0|
|       3|         1|         4|       0|      8|          4|      152.0|
|       4|         1|         2|       0|     14|          3|       87.0|
|       5|         1|         3|       0|     16|          6|      192.0|
|       6|         3|         1|       1|      4|         10|      190.0|
|       7|         2|         4|       2|     19|         10|      260.0|
|       8|         1|         0|       0|     14|          2|       90.0|
|       9|         3|         4|       1|      7|         10|      470.

### Exportação dos resultados

In [20]:
# dim_cliente.write.csv(os.path.join(OUTPUT_PATH,"dim_cliente.csv"), header=True, mode="overwrite")
# dim_produto.write.csv(os.path.join(OUTPUT_PATH,"dim_produto.csv"), header=True, mode="overwrite")
# dim_data.write.csv(os.path.join(OUTPUT_PATH,"dim_data.csv"), header=True, mode="overwrite")
# dim_local.write.csv(os.path.join(OUTPUT_PATH,"dim_local.csv"), header=True, mode="overwrite")

# fato_vendas.write.csv(os.path.join(OUTPUT_PATH,"fato_vendas.csv"), header=True, mode="overwrite")

In [21]:
# Encerrar a sessão
spark.stop()